In [ ]:
from torchreid.utils import FeatureExtractor

extractor = FeatureExtractor(
    model_name='osnet_ain_x1_0',
    model_path='./weights/osnet_ain_x1_0_msmt17_256x128_amsgrad_ep50_lr0.0015_coslr_b64_fb10_softmax_labsmth_flip_jitter.pth',
    device='cuda'
)

In [ ]:
from glob import glob
from pathlib import Path
TH = 0.7
def glob_(f):
    fs = glob(f)
    fs = [(Path(e).stem.split('_')[-1], e) for e in fs]
    #print('-------')
    #print(len(fs))
    fs = [e for c, e in fs if TH <= float(c)]
    #print(len(fs))
    return fs
    
ps = sorted(glob('reid_sample/*_'))
ps_ebds = [extractor(glob_(f'{p}/*.png')) for p in ps]